In [19]:
from langchain_community.llms import Ollama
llm= Ollama(model="llama3")

In [20]:
llm.invoke("how can langsmith help with testing?")

"Languishing is a great way to test and improve the quality of your language models. Here are some ways you can use Languishing to test your language model:\n\n1. **Grammar and syntax checks**: You can test your language model by inputting sentences or paragraphs that have errors in grammar, punctuation, or syntax. The model should flag these errors or suggest corrections.\n\n2. **Word choice and usage**: Test your language model's ability to recognize and use correct word choices by asking it to generate text on a specific topic. Check if the generated text uses proper vocabulary related to that topic.\n\n3. **Semantic coherence**: Languishing can help test whether your language model is capable of generating coherent, meaningful sentences. You can input a sentence or paragraph that requires semantic coherence and check if the model's output makes sense.\n\n4. **Contextual understanding**: Test your language model's ability to understand context by giving it a specific scenario or top

In [21]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world class technical documentation writer."),
    ("user","{input}")
])

In [22]:
chain = prompt | llm

In [23]:
chain.invoke({"input":"how can langsmith help with testing?"})

"As a world-class technical documentation writer, I'm excited to share my knowledge on how Langsmith can assist with testing.\n\nLangsmith is an AI-powered language processing platform that enables users to generate high-quality translations and summaries of technical content. When it comes to testing, Langsmith can help in the following ways:\n\n1. **Automated Translation Testing**: Langsmith's translation capabilities allow you to test your documentation against different language versions. You can translate a small portion of your document into multiple languages using Langsmith and then verify that the original intent is preserved across all translations.\n\n2. **Summary Generation for Test Scenarios**: Langsmith can generate concise summaries of complex technical content, making it easier to create test scenarios. By summarizing key points in a testable format, you can focus on verifying specific aspects of your documentation's accuracy and completeness.\n\n3. **Error Detection th

In [24]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [25]:
chain = prompt | llm | output_parser

In [26]:
chain.invoke({"input":"how can langsmith help with testing?"})

"As a world-class technical documentation writer, I'm excited to share some insights on how Langsmith can facilitate effective testing.\n\nLangsmith's AI-powered language generation capabilities can be leveraged in various ways to support testing efforts:\n\n1. **Test data generation**: Langsmith can generate test data that's relevant, realistic, and tailored to specific scenarios or conditions. This ensures that your tests are robust and cover a wide range of possibilities.\n2. **Automated test case creation**: By analyzing requirements and functional specifications, Langsmith can automatically create test cases, including preconditions, expected results, and test steps. This saves time and reduces the likelihood of human error in creating test cases.\n3. **Test data augmentation**: If you have limited or noisy test data, Langsmith can augment it by generating additional data that's similar in nature. This helps to increase the diversity and quantity of your test data.\n4. **Error mes

In [27]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [30]:
from langchain_community.embeddings  import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama3")

In [31]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [33]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [34]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"how can langsmith help with testing?",
    "context":[Document(page_content="langsmith can let you visualize test results")]
})

'According to the context, LangSmith can help with testing by allowing you to visualize test results.'

In [36]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [37]:
response = retrieval_chain.invoke({"input":"how can langsmith help with testing:"})
print(response['answer'])

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Tracing**: LangSmith provides native rendering of chat messages, functions, and retrieve documents, giving clear visibility and debugging information at each step of an LLM sequence.
2. **Debugging**: When things go wrong (unexpected end result, infinite agent loop, slower than expected execution, higher than expected token usage), Langsmith allows developers to debug by looking through application traces.
3. **Initial Test Set**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
4. **Comparison View**: LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of the application.
5. **Playground**: LangSmith's playground environment allows for rapid iteration and experimentation, making it easier to test out d

In [38]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an llm to generate this search query
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user","Given the above conversation,generate a search query to look up to get information relevant to the conversation")
])

retrievar_chain = create_history_aware_retriever(llm,retriever,prompt)

In [39]:
from langchain_core.messages import HumanMessage,AIMessage

chat_history= [HumanMessage(content="Can LangSmith help test my llm applications?"),AIMessage(content="Yes!")]

retriever_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my llm applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most i